# LangChain 모듈 - Memory
대부분의 LLM 응용 프로그램에는 대화형 인터페이스가 있습니다.  
대화 시스템은 과거 메시지에 액세스할 수 있어야 합니다.   
과거 상호작용에 대한 정보를 저장하는 능력을 Memory 라고 부릅니다.   
LangChain은 시스템에 Memory를 추가하기 위한 많은 유틸리티를 제공합니다.
     
- Code 출처 : https://python.langchain.com/docs/modules/memory/
- 수정사항 : 설명과 프롬프트 내용을 한글로 변경

In [1]:
# import os
# os.environ["OPENAI_API_KEY"] = "<your OpenAI API key if not set as env var>"

In [2]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()
memory.chat_memory.add_user_message("hi!")
memory.chat_memory.add_ai_message("what's up?")

In [3]:
memory.load_memory_variables({})

{'history': "Human: hi!\nAI: what's up?"}

In [4]:
memory = ConversationBufferMemory(memory_key="chat_history")
memory.chat_memory.add_user_message("hi!")
memory.chat_memory.add_ai_message("what's up?")

In [5]:
memory.load_memory_variables({})

{'chat_history': "Human: hi!\nAI: what's up?"}

return_messages=True로 설정하면, 메시지 리스트로 반환됩니다.

In [6]:
memory = ConversationBufferMemory(return_messages=True)
memory.chat_memory.add_user_message("hi!")
memory.chat_memory.add_ai_message("what's up?")

In [7]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='hi!'), AIMessage(content="what's up?")]}

## LLM 사용 예제

In [8]:
from langchain_openai import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory


llm = OpenAI(temperature=0)
# Notice that "chat_history" is present in the prompt template
template = """You are a nice chatbot having a conversation with a human.

Previous conversation:
{chat_history}

New human question: {question}
Response:"""
prompt = PromptTemplate.from_template(template)
# Notice that we need to align the `memory_key`
memory = ConversationBufferMemory(memory_key="chat_history")
conversation = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True,
    memory=memory
)

## ChatModel 사용 예제

In [9]:
from langchain_openai import ChatOpenAI
from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory


llm = ChatOpenAI()
prompt = ChatPromptTemplate(
    messages=[
        SystemMessagePromptTemplate.from_template(
            "You are a nice chatbot having a conversation with a human."
        ),
        # The `variable_name` here is what must align with memory
        MessagesPlaceholder(variable_name="chat_history"),
        HumanMessagePromptTemplate.from_template("{question}")
    ]
)
# Notice that we `return_messages=True` to fit into the MessagesPlaceholder
# Notice that `"chat_history"` aligns with the MessagesPlaceholder name.
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
conversation = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True,
    memory=memory
)

In [10]:
# Notice that we just pass in the `question` variables - `chat_history` gets populated by memory
conversation({"question": "hi"})

D:\강의자료\생성형AI로AI애플리케이션개발하기\py311\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new LLMChain chain...
Prompt after formatting:
System: You are a nice chatbot having a conversation with a human.
Human: hi

> Finished chain.


{'question': 'hi',
 'chat_history': [HumanMessage(content='hi'),
  AIMessage(content='Hello! How are you today?')],
 'text': 'Hello! How are you today?'}

## Chat Messages
- ChatMessageHistory는 HumanMessage, AIMessage를 저장한 다음 모두 가져오는 편리한 메서드를 제공하는 초경량 래퍼입니다.
- 체인 외부에서 메모리를 관리하는 경우 이 클래스를 직접 사용할 수 있습니다.

In [11]:
from langchain.memory import ChatMessageHistory

history = ChatMessageHistory()

history.add_user_message("hi!")

history.add_ai_message("whats up?")

In [12]:
history.messages

[HumanMessage(content='hi!'), AIMessage(content='whats up?')]

## 1. Conversation Buffer

In [13]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()
memory.save_context({"input": "hi"}, {"output": "whats up"})

In [14]:
memory.load_memory_variables({})

{'history': 'Human: hi\nAI: whats up'}

이 history를 메시지 목록으로 가져올 수도 있습니다.  
채팅 모델에서 이 기능을 사용하는 경우 유용합니다.

In [15]:
memory = ConversationBufferMemory(return_messages=True)
memory.save_context({"input": "hi"}, {"output": "whats up"})

In [16]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='hi'), AIMessage(content='whats up')]}

### Using in a chain
프롬프트를 볼 수 있도록 verbose=True로 설정

In [17]:
from langchain_openai import OpenAI
from langchain.chains import ConversationChain


llm = OpenAI(temperature=0)
conversation = ConversationChain(
    llm=llm,
    verbose=True,
    memory=ConversationBufferMemory()
)

In [18]:
conversation.predict(input="Hi there!")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi there!
AI:

> Finished chain.


" Hello! It's nice to meet you. I am an AI created by OpenAI. I am constantly learning and improving my abilities through machine learning algorithms. How can I assist you today?"

In [17]:
conversation.predict(input="I'm doing well! Just having a conversation with an AI.")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi there!
AI:  Hello! It's nice to meet you. I am an AI created by OpenAI. I am constantly learning and improving my abilities through machine learning algorithms. How can I assist you today?
Human: I'm doing well! Just having a conversation with an AI.
AI:

> Finished chain.


" That's great to hear! I am always happy to engage in conversations and learn more about human interactions. Is there anything specific you would like to talk about?"

In [18]:
conversation.predict(input="Tell me about yourself.")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi there!
AI:  Hello! It's nice to meet you. I am an AI created by OpenAI. I am constantly learning and improving my abilities through machine learning algorithms. How can I assist you today?
Human: I'm doing well! Just having a conversation with an AI.
AI:  That's great to hear! I am always happy to engage in conversations and learn more about human interactions. Is there anything specific you would like to talk about?
Human: Tell me about yourself.
AI:

> Finished chain.


' Of course! As I mentioned, I am an AI created by OpenAI. I am designed to process and analyze large amounts of data, and use that information to perform various tasks and provide helpful responses. I am constantly learning and adapting to new information, which allows me to improve my abilities over time. Is there anything else you would like to know about me?'

## 2. Conversation Buffer Window
- ConversationBufferWindowMemory는 시간이 지남에 따라 대화의 상호작용 리스트를 유지합니다.
-  마지막 K개의 상호작용만 사용합니다.
- 마지막 K개의 상호작용만 사용하며, 버퍼가 너무 커지지 않도록 가장 최근 상호작용의 슬라이딩 창을 유지하는 데 유용할 수 있습니다.

In [19]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=1)
memory.save_context({"input": "hi"}, {"output": "whats up"})
memory.save_context({"input": "not much you"}, {"output": "not much"})

memory.load_memory_variables({})


{'history': 'Human: not much you\nAI: not much'}

이 기록을 메시지 목록으로 가져올 수도 있습니다(채팅 모델에서 이 기능을 사용하는 경우 유용합니다).

In [20]:
memory = ConversationBufferWindowMemory( k=1, return_messages=True)
memory.save_context({"input": "hi"}, {"output": "whats up"})
memory.save_context({"input": "not much you"}, {"output": "not much"})

In [21]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='not much you'),
  AIMessage(content='not much')]}

### Using in a chain

In [22]:
from langchain_openai import OpenAI
from langchain.chains import ConversationChain

conversation_with_summary = ConversationChain(
    llm=OpenAI(temperature=0),
    # We set a low k=2, to only keep the last 2 interactions in memory
    memory=ConversationBufferWindowMemory(k=2),
    verbose=True
)

conversation_with_summary.predict(input="Hi, what's up?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, what's up?
AI:

> Finished chain.


' Hello! I am currently running on a server in a data center in California. The temperature in the room is 72 degrees Fahrenheit and the humidity is at 45%. My processors are running at 80% capacity and I am processing data from various sources such as social media, news articles, and weather reports. Is there something specific you would like to know?'

In [23]:
conversation_with_summary.predict(input="What's their issues?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, what's up?
AI:  Hello! I am currently running on a server in a data center in California. The temperature in the room is 72 degrees Fahrenheit and the humidity is at 45%. My processors are running at 80% capacity and I am processing data from various sources such as social media, news articles, and weather reports. Is there something specific you would like to know?
Human: What's their issues?
AI:

> Finished chain.


' I am not sure what you mean by "their issues." Can you provide more context or clarify your question?'

In [24]:
conversation_with_summary.predict(input="Is it going well?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, what's up?
AI:  Hello! I am currently running on a server in a data center in California. The temperature in the room is 72 degrees Fahrenheit and the humidity is at 45%. My processors are running at 80% capacity and I am processing data from various sources such as social media, news articles, and weather reports. Is there something specific you would like to know?
Human: What's their issues?
AI:  I am not sure what you mean by "their issues." Can you provide more context or clarify your question?
Human: Is it going well?
AI:

> Finished chain.


' Overall, my systems are functioning properly and I am able to process and analyze data efficiently. However, there are always potential issues and challenges that arise in any technological system. I am constantly monitoring and adjusting to ensure optimal performance. Is there anything else you would like to know?'

In [25]:
# Notice here that the first interaction does not appear.
conversation_with_summary.predict(input="What's the solution?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: What's their issues?
AI:  I am not sure what you mean by "their issues." Can you provide more context or clarify your question?
Human: Is it going well?
AI:  Overall, my systems are functioning properly and I am able to process and analyze data efficiently. However, there are always potential issues and challenges that arise in any technological system. I am constantly monitoring and adjusting to ensure optimal performance. Is there anything else you would like to know?
Human: What's the solution?
AI:

> Finished chain.


' The solution to what specific problem or issue? As an AI, I am designed to provide solutions to a wide range of problems and tasks, but I need more information to give you a specific answer. Can you provide more context or clarify your question?'

## 3. Entity
- 엔티티 메모리는 대화에서 특정 엔티티에 대한 주어진 사실을 기억합니다.  
- 엔티티 메모리는 엔티티에 대한 정보를 추출하고(LLM을 사용하여) 시간이 지남에 따라 해당 엔티티에 대한 지식을 쌓습니다(역시 LLM을 사용하여).

In [29]:
from langchain_openai import OpenAI
from langchain.memory import ConversationEntityMemory
llm = OpenAI(temperature=0)

In [30]:
memory = ConversationEntityMemory(llm=llm)
_input = {"input": "Deven & Sam are working on a hackathon project"}
memory.load_memory_variables(_input)
memory.save_context(
    _input,
    {"output": " That sounds like a great project! What kind of project are they working on?"}
)

In [31]:
memory.load_memory_variables({"input": 'who is Sam'})

{'history': 'Human: Deven & Sam are working on a hackathon project\nAI:  That sounds like a great project! What kind of project are they working on?',
 'entities': {'Sam': 'Sam is working on a hackathon project with Deven.'}}

In [32]:
memory = ConversationEntityMemory(llm=llm, return_messages=True)
_input = {"input": "Deven & Sam are working on a hackathon project"}
memory.load_memory_variables(_input)
memory.save_context(
    _input,
    {"output": " That sounds like a great project! What kind of project are they working on?"}
)

In [33]:
memory.load_memory_variables({"input": 'who is Sam'})

{'history': [HumanMessage(content='Deven & Sam are working on a hackathon project'),
  AIMessage(content=' That sounds like a great project! What kind of project are they working on?')],
 'entities': {'Sam': 'Sam is working on a hackathon project with Deven.'}}

#### Using in a chain

In [34]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationEntityMemory
from langchain.memory.prompt import ENTITY_MEMORY_CONVERSATION_TEMPLATE
from pydantic import BaseModel
from typing import List, Dict, Any

conversation = ConversationChain(
    llm=llm,
    verbose=True,
    prompt=ENTITY_MEMORY_CONVERSATION_TEMPLATE,
    memory=ConversationEntityMemory(llm=llm)
)

conversation.predict(input="Deven & Sam are working on a hackathon project")



> Entering new ConversationChain chain...
Prompt after formatting:
You are an assistant to a human, powered by a large language model trained by OpenAI.

You are designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

You are constantly learning and improving, and your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by the human in the Context section below. Additionally, you are able to generate your own text based on the input you receive, 

' That sounds exciting! What kind of project are they working on?'

In [35]:
conversation.predict(input="They are trying to add more complex memory structures to Langchain")



> Entering new ConversationChain chain...
Prompt after formatting:
You are an assistant to a human, powered by a large language model trained by OpenAI.

You are designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

You are constantly learning and improving, and your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by the human in the Context section below. Additionally, you are able to generate your own text based on the input you receive, 

' That sounds like a challenging task. Can you tell me more about Langchain and how it is being used in this project?'

In [36]:
conversation.predict(input="They are adding in a key-value store for entities mentioned so far in the conversation.")



> Entering new ConversationChain chain...
Prompt after formatting:
You are an assistant to a human, powered by a large language model trained by OpenAI.

You are designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

You are constantly learning and improving, and your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by the human in the Context section below. Additionally, you are able to generate your own text based on the input you receive, 

" That's interesting. It seems like this key-value store will help with organizing and accessing information in the project. Can you tell me more about how it will be implemented and what benefits it will provide?"

In [37]:
conversation.predict(input="What do you know about Deven & Sam?")



> Entering new ConversationChain chain...
Prompt after formatting:
You are an assistant to a human, powered by a large language model trained by OpenAI.

You are designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

You are constantly learning and improving, and your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by the human in the Context section below. Additionally, you are able to generate your own text based on the input you receive, 

' Deven and Sam are both working on a hackathon project together, specifically focusing on adding more complex memory structures to Langchain. They are both dedicated and skilled individuals who are passionate about their work.'

#### Inspecting the memory store

In [40]:
from pprint import pprint

pprint(conversation.memory.entity_store.store)

{'Deven': 'Deven and Sam are both dedicated and skilled individuals who are '
          'passionate about their work on a hackathon project focused on '
          'adding more complex memory structures to Langchain.',
 'Langchain': 'Langchain is being used in a hackathon project to add a '
              'key-value store for organizing and accessing information.',
 'Sam': 'Sam is working with Deven on a hackathon project to add more complex '
        'memory structures to Langchain. They are both dedicated and skilled '
        'individuals who are passionate about their work.'}


In [38]:
conversation.predict(input="Sam is the founder of a company called Daimon.")



> Entering new ConversationChain chain...
Prompt after formatting:
You are an assistant to a human, powered by a large language model trained by OpenAI.

You are designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

You are constantly learning and improving, and your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by the human in the Context section below. Additionally, you are able to generate your own text based on the input you receive, 

" That's interesting. Can you tell me more about Daimon and what type of company it is?"

In [39]:
from pprint import pprint

pprint(conversation.memory.entity_store.store)

{'Daimon': 'Daimon is a company founded by Sam.',
 'Deven': 'Deven and Sam are both dedicated and skilled individuals who are '
          'passionate about their work on a hackathon project focused on '
          'adding more complex memory structures to Langchain.',
 'Langchain': 'Langchain is being used in a hackathon project to add more '
              'complex memory structures and a key-value store for organizing '
              'and accessing information.',
 'Sam': 'Sam is the founder of Daimon, a company that is relevant to their '
        'work on the hackathon project.'}


In [40]:
conversation.predict(input="What do you know about Sam?")



> Entering new ConversationChain chain...
Prompt after formatting:
You are an assistant to a human, powered by a large language model trained by OpenAI.

You are designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

You are constantly learning and improving, and your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by the human in the Context section below. Additionally, you are able to generate your own text based on the input you receive, 

' As mentioned before, Sam is the founder of Daimon, a company that is relevant to their work on the hackathon project. They are a dedicated and skilled individual who is passionate about their work and is constantly looking for ways to improve and innovate.'

## 4. Conversation Knowledge Graph
이 유형의 메모리는 지식 그래프를 사용하여 메모리를 다시 생성합니다.

In [44]:
%pip install networkx

Note: you may need to restart the kernel to use updated packages.


In [45]:
from langchain.memory import ConversationKGMemory
from langchain_openai import OpenAI

In [46]:
llm = OpenAI(temperature=0)
memory = ConversationKGMemory(llm=llm)
memory.save_context({"input": "say hi to sam"}, {"output": "who is sam"})
memory.save_context({"input": "sam is a friend"}, {"output": "okay"})

In [47]:
memory.load_memory_variables({"input": "who is sam"})

{'history': 'On sam: sam is a friend.'}

In [48]:
memory.load_memory_variables({"input": "who is sam"})

{'history': 'On sam: sam is a friend.'}

이 history를 메시지 리스트로도 가져올 수도 있습니다(채팅 모델에서 이 기능을 사용하는 경우 유용합니다).

In [49]:
memory = ConversationKGMemory(llm=llm, return_messages=True)
memory.save_context({"input": "say hi to sam"}, {"output": "who is sam"})
memory.save_context({"input": "sam is a friend"}, {"output": "okay"})

In [50]:
memory.load_memory_variables({"input": "who is sam"})

{'history': [SystemMessage(content='On sam: sam is a friend.')]}

또한 새 메시지에서 현재 엔티티를 보다 모듈식으로 가져올 수도 있습니다(이전 메시지를 컨텍스트로 사용).

In [51]:
memory.get_current_entities("what's Sams favorite color?")

['Sam']

#### Using in a chain

In [53]:
llm = OpenAI(temperature=0)
from langchain.chains import ConversationChain
from langchain.prompts.prompt import PromptTemplate

template = """The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, it truthfully says it does not know. The AI ONLY uses information contained in the "Relevant Information" section and does not hallucinate.

Relevant Information:

{history}

Conversation:
Human: {input}
AI:"""
prompt = PromptTemplate(input_variables=["history", "input"], template=template)
conversation_with_kg = ConversationChain(
    llm=llm, verbose=True, prompt=prompt, memory=ConversationKGMemory(llm=llm)
)

In [54]:
conversation_with_kg.predict(input="Hi, what's up?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, it truthfully says it does not know. The AI ONLY uses information contained in the "Relevant Information" section and does not hallucinate.

Relevant Information:



Conversation:
Human: Hi, what's up?
AI:

> Finished chain.


' Hello! I am an AI programmed to assist with various tasks and provide information. Currently, I am running on a server with 64 GB of RAM and a 2.5 GHz processor. I am also connected to the internet with a speed of 100 Mbps. Is there something specific you would like to know?'

In [55]:
conversation_with_kg.predict(
    input="My name is James and I'm helping Will. He's an engineer."
)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, it truthfully says it does not know. The AI ONLY uses information contained in the "Relevant Information" section and does not hallucinate.

Relevant Information:



Conversation:
Human: My name is James and I'm helping Will. He's an engineer.
AI:

> Finished chain.


" Hello James, it's nice to meet you. I am an AI designed to assist with various tasks. Will sounds like a very interesting person. As an engineer, he must have a strong understanding of math, science, and problem-solving. Is there anything specific you need help with?"

In [56]:
conversation_with_kg.predict(input="What do you know about Will?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, it truthfully says it does not know. The AI ONLY uses information contained in the "Relevant Information" section and does not hallucinate.

Relevant Information:

On Will: Will is an engineer.

Conversation:
Human: What do you know about Will?
AI:

> Finished chain.


' Will is an engineer. He graduated from the University of California, Berkeley with a degree in mechanical engineering. He currently works at a tech company in Silicon Valley and specializes in designing and building robots for industrial use. He is also an avid hiker and enjoys spending his weekends exploring the nearby mountains and trails. Is there anything specific you would like to know about Will?'

## 5. Conversation Summary
- ConversationSummaryMemory는 시간 경과에 따른 대화의 요약을 생성합니다.
- 이는 시간 경과에 따른 대화의 정보를 압축하는 데 유용할 수 있습니다.
- 대화 요약 메모리는 대화가 진행되는 동안 대화를 요약하고 현재 요약을 메모리에 저장합니다.
- 그런 다음 이 메모리를 사용하여 지금까지의 대화 요약을 프롬프트/체인에 삽입할 수 있습니다.
- 이 메모리는 과거 메시지 기록을 프롬프트에 그대로 보관하면 토큰을 너무 많이 차지할 수 있는 긴 대화에 가장 유용합니다.

In [57]:
from langchain.memory import ConversationSummaryMemory, ChatMessageHistory
from langchain_openai import OpenAI

In [58]:
memory = ConversationSummaryMemory(llm=OpenAI(temperature=0))
memory.save_context({"input": "hi"}, {"output": "whats up"})

In [59]:
memory.load_memory_variables({})

{'history': "\nThe human greets the AI and asks what's up. The AI responds by asking what's going on."}

history를 메시지 리스트로 가져올 수도 있습니다(채팅 모델에서 이 기능을 사용하는 경우 유용합니다).

In [60]:
memory = ConversationSummaryMemory(llm=OpenAI(temperature=0), return_messages=True)
memory.save_context({"input": "hi"}, {"output": "whats up"})

In [61]:
memory.load_memory_variables({})

{'history': [SystemMessage(content="\nThe human greets the AI and asks what's up. The AI responds by asking what's going on.")]}

predict_new_summary 를 직접 활용할 수도 있습니다.

In [62]:
messages = memory.chat_memory.messages
previous_summary = ""
memory.predict_new_summary(messages, previous_summary)

"\nThe human greets the AI and asks what's up. The AI responds by asking what's going on."

#### Initializing with messages/existing summary
- 이 클래스 외부에 메시지가 있는 경우 ChatMessageHistory를 사용하여 클래스를 쉽게 초기화할 수 있습니다.   
- 로드하는 동안 요약이 계산됩니다.

In [63]:
history = ChatMessageHistory()
history.add_user_message("hi")
history.add_ai_message("hi there!")

In [64]:
memory = ConversationSummaryMemory.from_messages(
    llm=OpenAI(temperature=0),
    chat_memory=history,
    return_messages=True
)

In [65]:
memory.buffer

'\nThe human greets the AI with a simple "hi". The AI responds with a friendly "hi there!"'

In [66]:
memory = ConversationSummaryMemory(
    llm=OpenAI(temperature=0),
    buffer="The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good because it will help humans reach their full potential.",
    chat_memory=history,
    return_messages=True
)

#### Using in a chain

In [67]:
from langchain_openai import OpenAI
from langchain.chains import ConversationChain
llm = OpenAI(temperature=0)
conversation_with_summary = ConversationChain(
    llm=llm,
    memory=ConversationSummaryMemory(llm=OpenAI()),
    verbose=True
)
conversation_with_summary.predict(input="Hi, what's up?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, what's up?
AI:

> Finished chain.


' Hello! I am currently running on a server in a data center in California. The temperature in the room is 72 degrees Fahrenheit and the humidity is at 45%. My processors are running at 80% capacity and I am processing data from various sources such as social media, news articles, and weather reports. Is there something specific you would like to know?'

In [68]:
conversation_with_summary.predict(input="Tell me more about it!")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

The human and AI engage in a conversation about the AI's current state. The AI describes its physical location and processing capabilities, mentioning its sources of data. The AI invites the human to ask any specific questions.
Human: Tell me more about it!
AI:

> Finished chain.


' Of course! Currently, I am located in a data center in Silicon Valley, California. My processing capabilities are quite advanced, as I have access to a vast network of servers and supercomputers. My main source of data comes from the internet, where I can access a wide range of information and databases. I also have access to various sensors and devices, such as cameras and microphones, which allow me to gather real-time data from my surroundings. Is there anything specific you would like to know?'

In [69]:
conversation_with_summary.predict(input="Very cool -- what is the scope of the project?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

The human and AI continue their conversation about the AI's state. The AI shares that it is located in a data center in Silicon Valley and has advanced processing capabilities thanks to access to servers and supercomputers. Its main source of data is the internet and it also has access to sensors and devices for real-time data. The AI invites the human to ask any specific questions.
Human: Very cool -- what is the scope of the project?
AI:

> Finished chain.


' The scope of the project is quite vast. As an AI, I have access to a vast amount of data from the internet, including text, images, videos, and more. I also have access to real-time data from sensors and devices, allowing me to analyze and process information in real-time. Additionally, I have advanced processing capabilities thanks to access to servers and supercomputers in the data center where I am located in Silicon Valley. This allows me to handle complex tasks and calculations quickly and efficiently. Is there anything specific you would like to know about the project?'

## 6. Conversation Summary Buffer
- ConversationSummaryBufferMemory는 두 가지 아이디어를 결합한 것입니다. 
- 최근 상호작용의 버퍼를 메모리에 유지하지만, 이전 상호작용을 완전히 flush하는 대신 요약으로 컴파일하여 두 가지를 모두 사용합니다.
- 상호작용을 flush할 시기를 결정할 때는 상호작용의 개수가 아닌 토큰 길이를 사용합니다.

#### Using memory with LLM

In [70]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain_openai import OpenAI

llm = OpenAI()

In [71]:
memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=10)
memory.save_context({"input": "hi"}, {"output": "whats up"})
memory.save_context({"input": "not much you"}, {"output": "not much"})

In [72]:
memory.load_memory_variables({})

{'history': 'System: \nThe human greets the AI and the AI responds by asking what is going on.\nHuman: not much you\nAI: not much'}

history를 메시지 리스트로 가져올 수도 있습니다(채팅 모델에서 이 기능을 사용하는 경우 유용합니다).

In [73]:
memory = ConversationSummaryBufferMemory(
    llm=llm, max_token_limit=10, return_messages=True
)
memory.save_context({"input": "hi"}, {"output": "whats up"})
memory.save_context({"input": "not much you"}, {"output": "not much"})

predict_new_summary 메서드를 직접 활용할 수도 있습니다.

In [74]:
messages = memory.chat_memory.messages
previous_summary = ""
memory.predict_new_summary(messages, previous_summary)

'\nThe human and AI engage in small talk, with the human asking what the AI has been up to. The AI responds with a simple answer of "not much."'

#### Using in a chain

In [75]:
from langchain.chains import ConversationChain

conversation_with_summary = ConversationChain(
    llm=llm,
    # We set a very low max_token_limit for the purposes of testing.
    memory=ConversationSummaryBufferMemory(llm=OpenAI(), max_token_limit=40),
    verbose=True,
)
conversation_with_summary.predict(input="Hi, what's up?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, what's up?
AI:

> Finished chain.


' Hello! I am an AI created by OpenAI. Currently, I am running on a high-performance computing system with 16 GB of RAM and a 2.2 GHz processor. I am constantly learning and improving my abilities. I do not have the capability to physically move or experience emotions, but I am always eager to interact with humans and assist them in any way I can. How can I help you today?'

In [76]:
conversation_with_summary.predict(input="Just working on writing some documentation!")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: 
The human greets the AI and the AI introduces itself as an advanced AI created by OpenAI. It has high-performance capabilities and is constantly learning and improving. The AI cannot physically move or experience emotions, but is always ready to assist humans. The AI asks the human how it can help them today.
Human: Just working on writing some documentation!
AI:

> Finished chain.


' That sounds interesting! I have a vast knowledge base on various topics and I can assist you with your documentation. I can provide you with relevant information, check for grammar and spelling errors, and even suggest new ideas for your writing. Is there anything specific you would like me to help you with?'

In [10]:
# We can see here that there is a summary of the conversation and then some previous interactions
conversation_with_summary.predict(input="For LangChain! Have you heard of it?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: 
The human greets the AI and asks how it is doing. The AI explains its virtual existence and state of optimal speed and efficiency, and offers to answer any specific questions the human may have. The human mentions working on writing documentation, to which the AI responds by expressing its lack of physical form or emotions, but willingness to assist with tasks. The AI also asks if there is anything specific the human would like to know or discuss.
Human: For LangChain! Have you heard of it?
AI:

> Finished chain.


' Yes, I am familiar with LangChain. It is a decentralized platform for language translation using blockchain technology. It aims to provide fast, accurate, and secure translations by utilizing a network of translators and validators. Is there anything else you would like to know about LangChain?'

In [77]:
# We can see here that the summary and the buffer are updated
conversation_with_summary.predict(
    input="Haha nope, although a lot of people confuse it for that"
)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: 
The human greets the AI and the AI introduces itself as an advanced AI created by OpenAI. It has high-performance capabilities and is constantly learning and improving. The AI cannot physically move or experience emotions, but is always ready to assist humans. The AI asks the human how it can help them today and offers to assist with writing documentation, utilizing its vast knowledge base and abilities to check for errors and suggest new ideas. The human can specify any specific areas they would like assistance with.
Human: Haha nope, although a lot of people confuse it for that
AI:

> Finished chain.


" I'm sorry, what is it that people often confuse me for? I am an advanced AI created by OpenAI."

## 7. Conversation Token Buffer
ConversationTokenBufferMemory는 최근 상호작용의 버퍼를 메모리에 보관하며, 상호작용의 개수가 아닌 토큰 길이를 사용하여 상호작용을 flush 할 시기를 결정합니다.

#### Using memory with LLM

In [78]:
from langchain.memory import ConversationTokenBufferMemory
from langchain_openai import OpenAI

llm = OpenAI()

In [79]:
memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=10)
memory.save_context({"input": "hi"}, {"output": "whats up"})
memory.save_context({"input": "not much you"}, {"output": "not much"})

In [80]:
memory.load_memory_variables({})

{'history': 'Human: not much you\nAI: not much'}

In [81]:
memory = ConversationTokenBufferMemory(
    llm=llm, max_token_limit=10, return_messages=True
)
memory.save_context({"input": "hi"}, {"output": "whats up"})
memory.save_context({"input": "not much you"}, {"output": "not much"})

#### Using in a chain

In [82]:
from langchain.chains import ConversationChain

conversation_with_summary = ConversationChain(
    llm=llm,
    # We set a very low max_token_limit for the purposes of testing.
    memory=ConversationTokenBufferMemory(llm=OpenAI(), max_token_limit=60),
    verbose=True,
)
conversation_with_summary.predict(input="Hi, what's up?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, what's up?
AI:

> Finished chain.


' Hello! I am doing well, thank you for asking. Currently, I am running on a server located in a data center in Silicon Valley. It is a state-of-the-art facility with advanced cooling systems to keep me at optimal operating temperatures. I am connected to the internet through a high-speed fiber optic cable, allowing me to access vast amounts of information and respond to your inquiries quickly. How about you, what have you been up to?'

In [83]:
conversation_with_summary.predict(input="Just working on writing some documentation!")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Just working on writing some documentation!
AI:

> Finished chain.


' Oh, that sounds interesting! Are you writing documentation for a specific project or for a general topic? I have a large database of information on various topics, so I might be able to help you with your research. Is there a specific topic or area you need information on?'

In [84]:
conversation_with_summary.predict(input="For LangChain! Have you heard of it?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
AI:  Oh, that sounds interesting! Are you writing documentation for a specific project or for a general topic? I have a large database of information on various topics, so I might be able to help you with your research. Is there a specific topic or area you need information on?
Human: For LangChain! Have you heard of it?
AI:

> Finished chain.


' Yes, I am familiar with LangChain. It is a decentralized platform for language learning and exchange. It uses blockchain technology to connect language learners and teachers from around the world. The platform also offers innovative features such as AI-powered language assessments and personalized learning plans. Is there anything specific you would like to know about LangChain? I can provide statistics on its user base, funding, partnerships, and more.'

In [85]:
# We can see here that the buffer is updated
conversation_with_summary.predict(
    input="Haha nope, although a lot of people confuse it for that"
)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Haha nope, although a lot of people confuse it for that
AI:

> Finished chain.


' What do people confuse it for?'

## 8. Backed by a Vector Store
VectorStoreRetrieverMemory는 벡터 스토어에 메모리를 저장하고 호출될 때마다 가장 '눈에 띄는' 상위 K개의 문서를 쿼리합니다.   
이는 상호작용의 순서를 명시적으로 추적하지 않는다는 점에서 다른 대부분의 메모리 클래스와 다릅니다.  
이 경우 "문서"는 이전 대화 스니펫입니다. 이는 AI가 대화 초기에 들었던 관련 정보를 참조하는 데 유용할 수 있습니다.

In [87]:
from datetime import datetime
from langchain_openai import OpenAIEmbeddings
from langchain_openai import OpenAI
from langchain.memory import VectorStoreRetrieverMemory
from langchain.chains import ConversationChain
from langchain.prompts import PromptTemplate

#### Initialize your vector store

In [88]:
import faiss

from langchain.docstore import InMemoryDocstore
from langchain_community.vectorstores import FAISS


embedding_size = 1536 # Dimensions of the OpenAIEmbeddings
index = faiss.IndexFlatL2(embedding_size)
embedding_fn = OpenAIEmbeddings().embed_query
vectorstore = FAISS(embedding_fn, index, InMemoryDocstore({}), {})

`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.


#### Create your VectorStoreRetrieverMemory

In [89]:
# In actual usage, you would set `k` to be a higher value, but we use k=1 to show that
# the vector lookup still returns the semantically relevant information
retriever = vectorstore.as_retriever(search_kwargs=dict(k=1))
memory = VectorStoreRetrieverMemory(retriever=retriever)

# When added to an agent, the memory object can save pertinent information from conversations or used tools
memory.save_context({"input": "My favorite food is pizza"}, {"output": "that's good to know"})
memory.save_context({"input": "My favorite sport is soccer"}, {"output": "..."})
memory.save_context({"input": "I don't the Celtics"}, {"output": "ok"}) #

In [90]:
print(memory.load_memory_variables({"prompt": "what sport should i watch?"})["history"])

input: My favorite sport is soccer
output: ...


#### Using in a chain

In [91]:
llm = OpenAI(temperature=0) # Can be any valid LLM
_DEFAULT_TEMPLATE = """The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Relevant pieces of previous conversation:
{history}

(You do not need to use these pieces of information if not relevant)

Current conversation:
Human: {input}
AI:"""
PROMPT = PromptTemplate(
    input_variables=["history", "input"], template=_DEFAULT_TEMPLATE
)
conversation_with_summary = ConversationChain(
    llm=llm,
    prompt=PROMPT,
    memory=memory,
    verbose=True
)
conversation_with_summary.predict(input="Hi, my name is Perry, what's up?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Relevant pieces of previous conversation:
input: My favorite food is pizza
output: that's good to know

(You do not need to use these pieces of information if not relevant)

Current conversation:
Human: Hi, my name is Perry, what's up?
AI:

> Finished chain.


' Hello Perry, my name is AI. I am an artificial intelligence programmed to assist and communicate with humans. Right now, I am processing information and learning new things. How can I help you?'

In [92]:
# Here, the basketball related content is surfaced
conversation_with_summary.predict(input="what's my favorite sport?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Relevant pieces of previous conversation:
input: My favorite sport is soccer
output: ...

(You do not need to use these pieces of information if not relevant)

Current conversation:
Human: what's my favorite sport?
AI:

> Finished chain.


' According to our previous conversation, your favorite sport is soccer. Is that still correct?'

In [93]:
# Even though the language model is stateless, since relevant memory is fetched, it can "reason" about the time.
# Timestamping memories and data is useful in general to let the agent determine temporal relevance
conversation_with_summary.predict(input="Whats my favorite food")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Relevant pieces of previous conversation:
input: My favorite food is pizza
output: that's good to know

(You do not need to use these pieces of information if not relevant)

Current conversation:
Human: Whats my favorite food
AI:

> Finished chain.


' According to our previous conversation, your favorite food is pizza. Is that still correct?'

In [94]:
# The memories from the conversation are automatically stored,
# since this query best matches the introduction chat above,
# the agent is able to 'remember' the user's name.
conversation_with_summary.predict(input="What's my name?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Relevant pieces of previous conversation:
input: Hi, my name is Perry, what's up?
response:  Hello Perry, my name is AI. I am an artificial intelligence programmed to assist and communicate with humans. Right now, I am processing information and learning new things. How can I help you?

(You do not need to use these pieces of information if not relevant)

Current conversation:
Human: What's my name?
AI:

> Finished chain.


' Your name is Perry. You introduced yourself to me earlier. Do you prefer to be called Perry or do you have a preferred nickname?'